In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "citibike_tabular_data_28d.parquet")
df.head()



,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,2024-02-03,HB101
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-02-04,HB101
2,0,0,0,0,0,0,1,6,5,3,...,0,0,0,0,0,0,0,0,2024-02-05,HB101
3,4,5,4,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,2024-02-06,HB101
4,0,0,0,0,0,0,0,3,9,2,...,4,7,7,3,5,3,1,0,2024-02-07,HB101


In [4]:
df_est = df.copy()
df_est["pickup_hour"] = pd.to_datetime(df_est["pickup_hour"])
df_est["pickup_hour"] = df_est["pickup_hour"].dt.tz_localize("UTC").dt.tz_convert("America/New_York")


In [5]:
df.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,2024-02-03,HB101
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-02-04,HB101
2,0,0,0,0,0,0,1,6,5,3,...,0,0,0,0,0,0,0,0,2024-02-05,HB101
3,4,5,4,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,2024-02-06,HB101
4,0,0,0,0,0,0,0,3,9,2,...,4,7,7,3,5,3,1,0,2024-02-07,HB101


In [6]:
df_est.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,2024-02-02 19:00:00-05:00,HB101
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-02-03 19:00:00-05:00,HB101
2,0,0,0,0,0,0,1,6,5,3,...,0,0,0,0,0,0,0,0,2024-02-04 19:00:00-05:00,HB101
3,4,5,4,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,2024-02-05 19:00:00-05:00,HB101
4,0,0,0,0,0,0,0,3,9,2,...,4,7,7,3,5,3,1,0,2024-02-06 19:00:00-05:00,HB101


In [8]:
from datetime import datetime
from src.data_utils import split_time_series_data


X_train, y_train, X_test, y_test = split_time_series_data(
    df,  # or df_ts or your actual DataFrame
    cutoff_date=datetime(2025, 1, 1, 0, 0, 0),  # within your date range
    target_column="target"  # actual target column name
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(1665, 674)
(1665,)
(600, 674)
(600,)


In [9]:
import numpy as np

class BaselineModelPreviousHour:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]


In [10]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)


In [11]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


1.0833


In [12]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
load_dotenv() 

mlflow = set_mlflow_tracking()


INFO:src.experiment_utils:MLflow tracking URI and credentials set.


In [13]:
log_model_to_mlflow(
    model,
    X_test,
    "BaselineModelPreviousHour",
    "mean_absolute_error",
    score=test_mae
)


2025/05/10 04:57:50 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelPreviousHour' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousHour
INFO:src.experiment_utils:Logged mean_absolute_error: 1.0833333333333333
c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-mi

2025/05/10 04:58:02 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORKS_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Successfully registered model 'BaselineModelPreviousHour'.
2025/05/10 04:58:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelPreviousHour, version 1
Created version '1' of model 'BaselineModelPreviousHour'.
INFO:src.experiment_utils:Model logged with name: BaselineModelPreviousHour


🏃 View run upset-robin-489 at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/0/runs/3ff53c4bd4984d9bb232a362ff6b92ff
🧪 View experiment at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/0


In [14]:
class BaselineModelPreviousWeek:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*24}"]  # rides_t-168


In [15]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)


In [16]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

0.9950


In [17]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousWeek", "mean_absolute_error", score=test_mae)

2025/05/10 04:59:25 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelPreviousWeek' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousWeek
INFO:src.experiment_utils:Logged mean_absolute_error: 0.995
c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>

Successfully registered model 'BaselineModelPreviousWeek'.
2025/05/10 04:59:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelPreviousWeek, version 1
Created version '1' of model 'BaselineModelPreviousWeek'.
INFO:src.experiment_utils:Model logged with name: BaselineModelPreviousWeek


🏃 View run classy-quail-408 at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/1/runs/37761618be1d479eb72b93f2b25428c8
🧪 View experiment at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/1


In [18]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


0.9950


In [19]:
import numpy as np
import pandas as pd

class BaselineModelLast4Weeks:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass  # No training needed for baseline

    def predict(self, X_test: pd.DataFrame) -> np.array:
        last_4_weeks_columns = [
            f"rides_t-{7*24}",   # 1 week ago
            f"rides_t-{14*24}",  # 2 weeks ago
            f"rides_t-{21*24}",  # 3 weeks ago
            f"rides_t-{28*24}"   # 4 weeks ago
        ]
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")
        return X_test[last_4_weeks_columns].mean(axis=1).to_numpy()


In [20]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)


In [21]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


0.8817


In [22]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

0.8817


In [23]:
log_model_to_mlflow(model, X_test, "BaselineModelLast4Weeks", "mean_absolute_error", score=test_mae)


2025/05/10 05:00:06 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelLast4Weeks' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: BaselineModelLast4Weeks
INFO:src.experiment_utils:Logged mean_absolute_error: 0.8816666666666667
c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missin

Successfully registered model 'BaselineModelLast4Weeks'.
2025/05/10 05:00:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelLast4Weeks, version 1
Created version '1' of model 'BaselineModelLast4Weeks'.
INFO:src.experiment_utils:Model logged with name: BaselineModelLast4Weeks


🏃 View run bustling-yak-821 at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/2/runs/d2c11054ec93478c959e9fe03e94c762
🧪 View experiment at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/2


In [24]:
X_test.head()  # Get the total number of rows


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
333,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-01-01,HB101
334,0,0,0,0,0,0,0,0,0,0,...,1,0,4,1,2,1,1,3,2025-01-02,HB101
335,0,0,1,0,0,0,0,3,2,1,...,3,5,2,3,2,2,5,2,2025-01-03,HB101
336,0,0,0,0,0,0,0,0,0,0,...,0,7,6,0,0,1,1,3,2025-01-04,HB101
337,4,2,0,0,0,1,0,5,4,3,...,0,0,0,1,1,0,1,4,2025-01-05,HB101


In [45]:
plot_aggregated_time_series(X_test, y_test, row_id=220, predictions=predictions)
